In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import os, random

In [3]:
# Specify the directories and layer inresults_dirtion
results_dir = '.'

In [4]:
target_img_info = pd.read_csv((os.path.join(results_dir, 'selected_imgs_sEEG.csv')))

In [5]:
filler_img_info = pd.read_csv(os.path.join(results_dir, 'filler_imgs_sEEG.csv'))

In [6]:
# Specify the parameters
n_imgs_per_cond = 15
cond_dict = {1:'Large RE, Small Dist',
            2:'Large RE, Large Dist',
            3:'Small RE, Small Dist',
            4:'Small RE, Large Dist'}
trials_per_block = 150
total_blocks = 13

In [7]:
repeated_fillers = filler_img_info.iloc[:5*total_blocks]
nonrepeat_fillers = filler_img_info.iloc[5*total_blocks:].reset_index(drop=True)

In [8]:
nonrepeat_fillers

,Unnamed: 0,Index,Image type,Image index,Distinctiveness,Reconstruction error,Distinctiveness rank,RE rank
0,65,Target-1834,Target,1834,975.457656,0.044887,4656.0,4622.0
1,66,Target-1858,Target,1858,1016.735708,0.047515,5330.0,5351.0
2,67,Target-1861,Target,1861,971.274677,0.048557,4576.0,5611.0
3,68,Target-1874,Target,1874,968.407731,0.045017,4526.0,4664.0
4,69,Target-1903,Target,1903,978.725316,0.046014,4730.0,4918.0
...,...,...,...,...,...,...,...,...
354,419,Filler-7899,Filler,7899,1020.030821,0.044740,5389.0,4576.0
355,420,Filler-7919,Filler,7919,1000.305747,0.046899,5071.0,5178.0
356,421,Filler-7937,Filler,7937,1068.147854,0.048969,6230.0,5709.0
357,422,Filler-7982,Filler,7982,967.068730,0.046906,4511.0,5183.0


In [9]:
# Sort the images into different groups
Large_RE_Small_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Small')].sort_values(by='Reconstruction error', ascending=False).reset_index(drop=True)
Large_RE_Large_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Large')].sort_values(by='Reconstruction error', ascending=False).reset_index(drop=True)
Small_RE_Small_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Small')].sort_values(by='Reconstruction error').reset_index(drop=True)
Small_RE_Large_Dist = target_img_info[(target_img_info['RE group']=='Large')&
                                     (target_img_info['Dist group']=='Large')].sort_values(by='Reconstruction error').reset_index(drop=True)

In [10]:
Large_RE_Small_Dist

,Unnamed: 0,Index,Image type,Image index,Distinctiveness,Reconstruction error,Distinctiveness rank,RE rank,RE group,Dist group,Rank diff,Rank sum
0,167,Filler-6188,Filler,6188,535.458801,0.106418,564.5,10278.0,Large,Small,NaN,NaN
1,41,Target-1458,Target,1458,535.458801,0.105238,564.5,10265.0,Large,Small,NaN,NaN
2,10,Target-270,Target,270,801.236621,0.104338,2383.0,10259.0,Large,Small,NaN,NaN
3,174,Filler-6435,Filler,6435,604.342811,0.102195,900.0,10241.0,Large,Small,NaN,NaN
4,131,Filler-4507,Filler,4507,777.991300,0.097216,2150.0,10175.0,Large,Small,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
190,187,Filler-7517,Filler,7517,716.469427,0.058820,1607.0,7850.0,Large,Small,NaN,NaN
191,190,Filler-7648,Filler,7648,737.341632,0.058792,1784.0,7846.0,Large,Small,NaN,NaN
192,50,Target-1988,Target,1988,796.959918,0.058760,2334.0,7841.0,Large,Small,NaN,NaN
193,103,Filler-2269,Filler,2269,736.400173,0.058728,1774.0,7835.0,Large,Small,NaN,NaN


In [11]:
img_cond_dict = {1:Large_RE_Small_Dist,
                2:Large_RE_Large_Dist,
                3:Small_RE_Small_Dist,
                4:Small_RE_Large_Dist}

1. Memorability effect is quite robust at different memory lags. Maybe let's just try to repeat the target images at lages of 20-40. We should make sure that the mean repetition lag doesn't differ across the four groups of images.
2. We can have filler images (i.e. those that are not part of the four groups) that repeat at a shorter timescale (1-3), as an attention check.
3. Because it may be tiring to do the task all in one go, we can break the task in to 4 blocks of 7 minutes each.
Each block consists of 17 images from each of the target image group (therefore 68 in total), 5 filler images that repeat at short timescle, and 30 fller images that never repeat. This leads to a total of 176 trials each.
4. Following Lin et al. (2021, Cognition), each image will be presented for 1.2 s and ITI is 1.2
Therefore, each block will last about 7 mins (422.4 seconds).
5. Filler images can be sampled from the filler_images dataframe, which are images with RE and Dist in the middle.
The experiment dataframe should consist of the following columns: block number, trial number, image index, trial type (old or new), response (empty for now) and reaction time (empty for now)

In [12]:
def generate_trial_sequence(
    num_conditions=4,
    images_per_condition=15,
    target_repeat_lag_range=(20, 40),
    num_filler_single=20,
    num_filler_repeat=5,
    filler_repeat_lag_range=(1, 3),
    total_trials=150,
    seed=None,
    max_repeats_in_last_n=6,
    check_last_n=10
):
    if seed is not None:
        random.seed(seed)

    # 1. Generate image labels
    target_images = [
        f'T{cond}_{i}' for cond in range(1, num_conditions + 1)
        for i in range(1, images_per_condition + 1)
    ]
    filler_single = [f'F_S{i}' for i in range(1, num_filler_single + 1)]
    filler_repeat = [f'F_R{i}' for i in range(1, num_filler_repeat + 1)]

    sequence = [None] * total_trials
    used_positions = set()

    # ---------- new：record img_pairs & single_filter' positions ----------
    repeat_pairs = {}          # img -> (first_idx, second_idx)
    single_positions = []      # 所有 single-filler 的 idx（只记一次）

    def place_with_lag(img, min_lag, max_lag, total_len):
        attempts = 0
        while attempts < 1000:
            pos1 = random.randint(0, total_len - min_lag - 1)
            lag = random.randint(min_lag, max_lag)
            pos2 = pos1 + lag
            if pos2 >= total_len:
                attempts += 1
                continue
            if pos1 not in used_positions and pos2 not in used_positions:
                return pos1, pos2
            attempts += 1
        return None, None

    # 2. Place target images
    for img in target_images:
        pos1, pos2 = place_with_lag(img, *target_repeat_lag_range, total_trials)
        if pos1 is None:
            raise RuntimeError(f"Could not place target {img}")
        sequence[pos1] = (img, False)
        sequence[pos2] = (img, True)
        used_positions.update([pos1, pos2])
        ##---------- new：record repeat_img_pairs' positions ----------
        repeat_pairs[img] = (pos1, pos2)

    # 3. Place repeating fillers
    for img in filler_repeat:
        pos1, pos2 = place_with_lag(img, *filler_repeat_lag_range, total_trials)
        if pos1 is None:
            raise RuntimeError(f"Could not place repeating filler {img}")
        sequence[pos1] = (img, False)
        sequence[pos2] = (img, True)
        used_positions.update([pos1, pos2])
        ##---------- new：record repeat_img_pairs' positions ----------
        repeat_pairs[img] = (pos1, pos2)

    # 4. Place non-repeating fillers
    remaining_slots = [i for i in range(total_trials) if i not in used_positions]
    if len(remaining_slots) < num_filler_single:
        raise RuntimeError("Not enough slots for single fillers")
    random.shuffle(remaining_slots)


    for i, img in enumerate(filler_single):
        pos = remaining_slots[i]
        sequence[pos] = (img, False)
        used_positions.add(pos)
        ##----------- record single filter's position ----------
        single_positions.append(pos) ## record single filter's position

    # 5. Post-hoc check: Too many repeats at the end?
    def count_repeats(trial_list, start):
        return sum(1 for t in trial_list[start:] if t and t[1] is True)

    last_n_start = total_trials - check_last_n
    repeat_count = count_repeats(sequence, last_n_start)

    ##---------------------new-------------------------------------------
    def move_tail_repeats_with_single(sequence, single_pos, repeat_pairs, max_repeats_in_last_n, check_last_n, total = total_trials):
        last_start = total - check_last_n

        tail_second = [i for i in range(last_start, total) if sequence[i] and sequence[i][1] is True]
        need = len(tail_second) - max_repeats_in_last_n

        # pick 0-139  single filler
        candidates = sorted([i for i in single_pos if i < last_start and sequence[i]])
        for _ in range(need):
            if not candidates or not tail_second:
                break
            dst_second = tail_second.pop()
            src_single = candidates.pop(0)
            img = sequence[dst_second][0]
            first_repeat = repeat_pairs[img][0]

            # =====after swap need make sure: first ≤ second =====
            if src_single > first_repeat:
                sequence[src_single], sequence[dst_second] = sequence[dst_second], sequence[src_single]
                repeat_pairs[img] = (first_repeat, src_single)

            else:
                sequence[src_single], sequence[dst_second] = sequence[dst_second], sequence[src_single]
                new_first_repeat = src_single
                new_second_repeat = first_repeat
                repeat_pairs[img] = (new_first_repeat , new_second_repeat)
                sequence[new_first_repeat] = (img, False)
                sequence[new_second_repeat] = (img, True)
                continue


    if repeat_count > max_repeats_in_last_n:
        move_tail_repeats_with_single(
            sequence, single_positions, repeat_pairs,
            max_repeats_in_last_n, check_last_n, total_trials)


    # if repeat_count > max_repeats_in_last_n:
    #     # Try to reduce by swapping
    #     early_idxs = [i for i in range(total_trials - check_last_n) if sequence[i] and sequence[i][1] is False]
    #     late_idxs = [i for i in range(last_n_start, total_trials) if sequence[i] and sequence[i][1] is True]
    #
    #     swaps_done = 0
    #     for late_idx in late_idxs:
    #         for early_idx in early_idxs:
    #             # Swap early non-repeat with late repeat
    #             sequence[early_idx], sequence[late_idx] = sequence[late_idx], sequence[early_idx]
    #             new_repeat_count = count_repeats(sequence, last_n_start)
    #             if new_repeat_count <= max_repeats_in_last_n:
    #                 swaps_done += 1
    #                 break
    #             else:
    #                 # Revert and try next
    #                 sequence[early_idx], sequence[late_idx] = sequence[late_idx], sequence[early_idx]
    #         if count_repeats(sequence, last_n_start) <= max_repeats_in_last_n:
    #             break

    # Final check
    if count_repeats(sequence, last_n_start) > max_repeats_in_last_n:
        raise RuntimeError("Could not enforce repeat constraint in last trials")

    # 6. Format output
    final_sequence = []
    for i, trial in enumerate(sequence):
        if trial is None:
            final_sequence.append({
                'trial_num': i + 1,
                'image': None,
                'is_repeat': None,
                'type': 'EMPTY'
            })
        else:
            img, is_repeat = trial
            trial_type = 'target' if img.startswith('T') else 'filler'
            final_sequence.append({
                'trial_num': i + 1,
                'image': img,
                'is_repeat': is_repeat,
                'type': trial_type
            })

    return final_sequence


In [13]:
def generate_valid_sequence(max_attempts=100, verbose=True):
    for attempt in range(max_attempts):
        seed = random.randint(0, 1000000)
        try:
            sequence = generate_trial_sequence(seed=seed)
            if verbose:
                print(f"✅ Sequence generated successfully on attempt {attempt+1} (seed={seed})")
            return sequence
        except RuntimeError as e:
            if verbose:
                print(f"⚠️ Attempt {attempt+1} failed (seed={seed}): {str(e)}")
    raise RuntimeError("❌ Failed to generate a valid sequence after multiple attempts.")


In [14]:
for curr_block in range(total_blocks):
    # Generate trial sequence
    
    block_sequence_dict = generate_valid_sequence()
    
    # Place the images
    if curr_block != 12:
        df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
        df_f_single = nonrepeat_fillers.iloc[curr_block*20:(curr_block+1)*20]
        df_f_repeat = repeated_fillers.iloc[curr_block*5:(curr_block+1)*5]
    else:
        df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:]
        df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:]
        df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:]
        df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:]
        df_f_single = nonrepeat_fillers.iloc[curr_block*20:]
        df_f_repeat = repeated_fillers.iloc[curr_block*5:]
    
    # Step 1: Build a lookup dictionary for images
    image_lookup = {}
    # Target conditions
    for i, df in enumerate([df_t1, df_t2, df_t3, df_t4], start=1):
        for j, path in enumerate(df['Index']):
            label = f'T{i}_{j+1}'  # matches label from generate_trial_sequence
            image_lookup[label] = path

    # Fillers - single appearance
    for i, path in enumerate(df_f_single['Index'], start=1):
        label = f'F_S{i}'
        image_lookup[label] = path

    # Fillers - repeating
    for i, path in enumerate(df_f_repeat['Index'], start=1):
        label = f'F_R{i}'
        image_lookup[label] = path

    # Step 2: Assign image paths to your generated sequence

    for trial in block_sequence_dict:
        label = trial['image']
        trial['Image Index'] = image_lookup.get(label, None)
    
    block_df = pd.DataFrame(block_sequence_dict)
    block_df['Block'] = curr_block+1


✅ Sequence generated successfully on attempt 1 (seed=994372)
✅ Sequence generated successfully on attempt 1 (seed=91503)
✅ Sequence generated successfully on attempt 1 (seed=182177)
✅ Sequence generated successfully on attempt 1 (seed=934134)
✅ Sequence generated successfully on attempt 1 (seed=499715)
✅ Sequence generated successfully on attempt 1 (seed=815260)
✅ Sequence generated successfully on attempt 1 (seed=711984)
✅ Sequence generated successfully on attempt 1 (seed=593573)
✅ Sequence generated successfully on attempt 1 (seed=829160)
✅ Sequence generated successfully on attempt 1 (seed=463831)
✅ Sequence generated successfully on attempt 1 (seed=852748)
✅ Sequence generated successfully on attempt 1 (seed=947763)
✅ Sequence generated successfully on attempt 1 (seed=725611)


In [15]:
NB_dir   = os.getcwd()
PROJECT  = os.path.dirname(os.path.dirname(NB_dir))
# 实验数据目录
Exp_dir  = os.path.join(PROJECT, 'Experiment/')

os.makedirs(Exp_dir, exist_ok=True)
print("experiment_design directory：", Exp_dir)


experiment_design directory： /Users/xuezhang/Documents/SKKU/ReconMem_cont_ecog/Experiment/


In [17]:
for sub in range(1, 51):
    sub_dir = os.path.join(Exp_dir, f"sub-{sub:02d}")
    if not os.path.isdir(sub_dir):
        os.mkdir(sub_dir)
    for curr_block in range(total_blocks):
        # Generate trial sequence
        block_sequence_dict = generate_valid_sequence()

        # Place the images
        if curr_block != 12:
            df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:(curr_block+1)*n_imgs_per_cond]
            df_f_single = nonrepeat_fillers.iloc[curr_block*20:(curr_block+1)*20]
            df_f_repeat = repeated_fillers.iloc[curr_block*5:(curr_block+1)*5]
        else:
            df_t1 = img_cond_dict[1].iloc[curr_block*n_imgs_per_cond:]
            df_t2 = img_cond_dict[2].iloc[curr_block*n_imgs_per_cond:]
            df_t3 = img_cond_dict[3].iloc[curr_block*n_imgs_per_cond:]
            df_t4 = img_cond_dict[4].iloc[curr_block*n_imgs_per_cond:]
            df_f_single = nonrepeat_fillers.iloc[curr_block*20:]
            df_f_repeat = repeated_fillers.iloc[curr_block*5:]

        # Step 1: Build a lookup dictionary for images
        image_lookup = {}
        # Target conditions
        for i, df in enumerate([df_t1, df_t2, df_t3, df_t4], start=1):
            for j, path in enumerate(df['Index']):
                label = f'T{i}_{j+1}'  # matches label from generate_trial_sequence
                image_lookup[label] = path

        # Fillers - single appearance
        for i, path in enumerate(df_f_single['Index'], start=1):
            label = f'F_S{i}'
            image_lookup[label] = path

        # Fillers - repeating
        for i, path in enumerate(df_f_repeat['Index'], start=1):
            label = f'F_R{i}'
            image_lookup[label] = path

        # Step 2: Assign image paths to your generated sequence

        for trial in block_sequence_dict:
            label = trial['image']
            trial['Image Index'] = image_lookup.get(label, None)

        block_df = pd.DataFrame(block_sequence_dict)
        block_df['Block'] = curr_block+1
        
        block_dir = os.path.join(sub_dir, f"Block{curr_block+1:02d}_info.csv")
        block_df.to_csv(block_dir)

✅ Sequence generated successfully on attempt 1 (seed=845431)
✅ Sequence generated successfully on attempt 1 (seed=471842)
✅ Sequence generated successfully on attempt 1 (seed=660515)
✅ Sequence generated successfully on attempt 1 (seed=848018)
✅ Sequence generated successfully on attempt 1 (seed=61333)
✅ Sequence generated successfully on attempt 1 (seed=753699)
✅ Sequence generated successfully on attempt 1 (seed=477407)
✅ Sequence generated successfully on attempt 1 (seed=935540)
✅ Sequence generated successfully on attempt 1 (seed=205867)
✅ Sequence generated successfully on attempt 1 (seed=620230)
✅ Sequence generated successfully on attempt 1 (seed=7064)
✅ Sequence generated successfully on attempt 1 (seed=572555)
✅ Sequence generated successfully on attempt 1 (seed=513090)
✅ Sequence generated successfully on attempt 1 (seed=971951)
✅ Sequence generated successfully on attempt 1 (seed=941840)
✅ Sequence generated successfully on attempt 1 (seed=145909)
✅ Sequence generated succes

In [18]:
block1_df = pd.read_csv('../../Experiment/sub-01/Block01_info.csv')

In [19]:
block13_df = pd.read_csv('../../Experiment/sub-01/Block02_info.csv')


In [20]:
np.sum(block1_df['Image Index'].isin(block13_df['Image Index']))

np.int64(0)